## Train an AspectEmo-HerBERT model

inspired by
https://www.freecodecamp.org/news/getting-started-with-ner-models-using-huggingface/

search for TODO_INSERT_YOUR_OWN and replace with your content (eg. paths)

In [ ]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --upgrade sacremoses
!pip install --upgrade transformers
!pip install --upgrade seqeval
!pip install --upgrade datasets
!pip install --upgrade tokenizers
!pip install --upgrade wandb
!pip install cloud-tpu-client==0.10 torch==1.11.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.11-cp37-cp37m-linux_x86_64.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 880 kB 5.3 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=42bf7e3dc14a7d5a8b0fea3b29d307c5b5c7245904a03f7a7553a9a6be0fce31
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.3 MB/s 
     |████████████████████████████████| 84 kB 3.5 MB/s 
     |████████████████████████████████| 6.6 MB 52.4 MB/s 
     |████████████████████████████████| 596 kB 72.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev

Let's convert to Transformers
eg. https://stackoverflow.com/questions/71618974/convert-pandas-dataframe-to-datasetdict

In [ ]:
import codecs, json
import pandas as pd
import math
import numpy as np
from tqdm import tqdm,trange
import datasets

datapath = TODO_INSERT_YOUR_OWN
suffix = 'train' # change to train if needed

with open(datapath+'data.json', 'r') as f:
  polemo_json = json.load(f)

all_tags = []
for fname in polemo_json['train'].keys():
  all_tags.extend( polemo_json['train'][fname]['labels'] )

unique_tags = set(all_tags)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}
label_names = list(tag2id.keys())

def get_toks_and_tags(split):
  texts = {}
  texts['tokens'] = []
  texts['labels'] = []
  for fname in polemo_json[split].keys():
    texts['tokens'].append( polemo_json[split][fname]['tokens'] )
    texts['labels'].append( [tag2id[x] for x in polemo_json[split][fname]['labels'] ] )

  return datasets.Dataset.from_dict(texts)

dataset = datasets.DatasetDict({"train":get_toks_and_tags('train'),"test":get_toks_and_tags('test')})

In [ ]:
from transformers import (
    BertConfig,
    BertTokenizer,
    AutoTokenizer,
    BertForTokenClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    create_optimizer)

BERT_MODEL="allegro/herbert-base-cased"

MODEL_CLASSES = {"bert": (BertConfig, BertForTokenClassification, AutoTokenizer)}
config_class, model_class, tokenizer_class = MODEL_CLASSES['bert']

config = config_class.from_pretrained(BERT_MODEL, num_labels=len(id2tag), id2label=id2tag, label2id=tag2id ) #, output_attentions=True)
model = model_class.from_pretrained(
                BERT_MODEL,
                config=config)

tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL, do_lower_case=False,  padding='max_length')

Downloading:   0%|          | 0.00/472 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/886k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/543k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/129 [00:00<?, ?B/s]

Method `tokenize_adjust_labels` for preparing data: tokenization to the format required by transformers, taking into account tagging at the token level (filling with a special tag -100 consecutive tokens of a given word).

In [ ]:
#Get the values for input_ids, token_type_ids, attention_mask
def tokenize_adjust_labels(all_samples_per_split):
  tokenized_samples = tokenizer.batch_encode_plus(all_samples_per_split["tokens"], is_split_into_words=True, truncation=True, padding='max_length')
  #tokenized_samples is not a datasets object so this alone won't work with Trainer API, hence map is used
  #so the new keys [input_ids, labels (after adjustment)]
  #can be added to the datasets dict for each train test validation split
  total_adjusted_labels = []
  for k in range(0, len(tokenized_samples["input_ids"])):
    prev_wid = -1
    word_ids_list = tokenized_samples.word_ids(batch_index=k)
    existing_label_ids = all_samples_per_split["labels"][k]
    i = -1
    adjusted_label_ids = []

    for wid in word_ids_list:
      if(wid is None):
        adjusted_label_ids.append(-100)
      elif(wid!=prev_wid):
        i = i + 1
        adjusted_label_ids.append(existing_label_ids[i])
        prev_wid = wid
      else:
        adjusted_label_ids.append(existing_label_ids[i])

    total_adjusted_labels.append(adjusted_label_ids)
  tokenized_samples["labels"] = total_adjusted_labels
  return tokenized_samples

tokenized_dataset = dataset.map(tokenize_adjust_labels, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, EvalPrediction

import numpy as np
from datasets import load_metric

metric = load_metric("seqeval")

def compute_metrics(p: EvalPrediction):
    predictions = p.predictions
    labels = p.label_ids
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    flattened_results = {
        "overall_precision": results["overall_precision"],
        "overall_recall": results["overall_recall"],
        "overall_f1": results["overall_f1"],
        "overall_accuracy": results["overall_accuracy"],
    }
    for k in results.keys():
      if(k not in flattened_results.keys()):
        flattened_results[k+"_f1"]=results[k]["f1"]

    return flattened_results

In [ ]:
import logging
logger = logging.getLogger("wandb")
logger.setLevel(logging.ERROR)

Train the model:

In [ ]:
import os
import wandb
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer, padding='max_length')


training_args = TrainingArguments(
    output_dir="./fine_tune_herbert_output",
    overwrite_output_dir = True,
    do_eval=True,
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    tpu_num_cores=8,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    adam_epsilon=1e-10,
    logging_strategy="epoch",
    run_name = "aspectemo_run1",
    save_strategy='epoch'
    #load_best_model_at_end=True,
    #save_total_limit = 1
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
wandb.finish()

Write the best checkpoint:

In [ ]:
!cp -r ./fine_tune_herbert_output/checkpoint-444 TODO_INSERT_YOUR_OWN



---
# Use the previously-prepared AspectEmo-HerBERT model:


In [ ]:
from transformers import (
    BertConfig,
    BertTokenizer,
    AutoTokenizer,
    BertForTokenClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    create_optimizer)

model_directory = TODO_INSERT_YOUR_OWN
BERT_MODEL="allegro/herbert-base-cased"

MODEL_CLASSES = {"bert": (BertConfig, BertForTokenClassification, AutoTokenizer)}
config_class, model_class, tokenizer_class = MODEL_CLASSES['bert']

config = config_class.from_pretrained(BERT_MODEL, num_labels=len(id2tag), id2label=id2tag, label2id=tag2id )
aspect_model = model_class.from_pretrained(model_directory, config=config)

tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL, do_lower_case=False,  padding='max_length')

in the below cell, set TODO_INSERT_YOUR_OWN to your path to excel sheet with target domain samples:

In [ ]:
import pandas as pd

silownie_df = pd.read_excel(TODO_INSERT_YOUR_OWN+'/few_shot_aspectemo.xlsx', sheet_name='najlepsze-silownie.pl')
filmweb_df = pd.read_excel(TODO_INSERT_YOUR_OWN+'/few_shot_aspectemo.xlsx', sheet_name='filmweb')
restauracje_df = pd.read_excel(TODO_INSERT_YOUR_OWN+'/few_shot_aspectemo.xlsx', sheet_name='restauracje')


### zero-shot method

Prediction using the few-shot on the model AspectEmo-HerBERT:

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer, padding='max_length')
trainer = Trainer(model=aspect_model, data_collator=data_collator, tokenizer=tokenizer, compute_metrics=compute_metrics)

all_results = {}
for domena, df in {'silownie':silownie_df, 'filmweb':filmweb_df, 'restauracje':restauracje_df}.items():

  print(domena)
  all_results[domena] = []

  texts = {}
  texts['tokens'] = []
  texts['labels'] = []
  toks = []; labels = []
  for index, row in df.iterrows():
    if pd.isna(row['token']):
      if len(toks)>0:
        assert len(labels) == len(toks)
        texts['tokens'].append( toks )
        texts['labels'].append( labels )
      toks = []; labels = []
    else:
      toks.append( str(row['token']).strip() )
      labels.append( tag2id['O'] if pd.isna(row['label']) else tag2id[ str(row['label']).strip() ] )

  few_shot_dataset = datasets.Dataset.from_dict(texts).map(tokenize_adjust_labels, batched=True)
  predictions = trainer.predict(few_shot_dataset)
  print(compute_metrics(predictions))

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


silownie


  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 29
  Batch size = 8


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: a_plus_m seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: a_plus_s seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: a_amb seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: a_zero seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: a_minus_s seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_label

...

The following columns in the test set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 29
  Batch size = 8


{'overall_precision': 0.03125, 'overall_recall': 0.06493506493506493, 'overall_f1': 0.04219409282700422, 'overall_accuracy': 0.018315018315018316, '_amb_f1': 0.024691358024691357, '_minus_m_f1': 0.0, '_minus_s_f1': 0.046511627906976744, '_plus_m_f1': 0.0, '_plus_s_f1': 0.0, '_zero_f1': 0.27586206896551724}
restauracje


  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 29
  Batch size = 8


{'overall_precision': 0.022222222222222223, 'overall_recall': 0.06321839080459771, 'overall_f1': 0.03288490284005979, 'overall_accuracy': 0.018756169792694965, '_amb_f1': 0.029197080291970802, '_minus_m_f1': 0.0, '_minus_s_f1': 0.0, '_plus_m_f1': 0.0, '_plus_s_f1': 0.0, '_zero_f1': 0.2571428571428571}


### For comparison, a few-shot approach using classic gradient-based training

In [ ]:
from transformers import DataCollatorForTokenClassification
import os

data_collator = DataCollatorForTokenClassification(tokenizer, padding='max_length')

os.environ["WANDB_DISABLED"] = "true"

all_results = {}
for domena, df in {'silownie':silownie_df, 'filmweb':filmweb_df, 'restauracje':restauracje_df}.items():

  print(f"{domena} --------------------")
  all_results[domena] = []

  texts = {}
  texts['tokens'] = []
  texts['labels'] = []
  toks = []; labels = []
  for index, row in df.iterrows():
    if pd.isna(row['token']):
      if len(toks)>0:
        assert len(labels) == len(toks)
        texts['tokens'].append( toks )
        texts['labels'].append( labels )
      toks = []; labels = []
    else:
      toks.append( str(row['token']).strip() )
      labels.append( tag2id['O'] if pd.isna(row['label']) else tag2id[ str(row['label']).strip() ] )

  samples = []
  for _ in range(3):
    idx = np.random.choice(len(texts['tokens']), 20, replace=False)
    s_idx, q_idx = np.split(idx, 2)
    texts_q = {} ; texts_s = {}
    texts_q['tokens'] = [texts['tokens'][i] for i in q_idx]
    texts_q['labels'] = [texts['labels'][i] for i in q_idx]

    texts_s['tokens'] = [texts['tokens'][i] for i in s_idx]
    texts_s['labels'] = [texts['labels'][i] for i in s_idx]

    dataset = datasets.DatasetDict({"s": datasets.Dataset.from_dict(texts_s),"q":datasets.Dataset.from_dict(texts_q)})
    tokenized_dataset_q_s = dataset.map(tokenize_adjust_labels, batched=True)

    # ladowanie AspectEmo-HerBERT od nowa dla kazdej domeny
    aspect_model = model_class.from_pretrained(model_directory, config=config)

    training_args = TrainingArguments(
        do_eval=True,
        learning_rate=5e-5,
        tpu_num_cores=8,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        #weight_decay=0.01,
        adam_epsilon=1e-10,
        evaluation_strategy="epoch",
        output_dir="tmp"
        )

    trainer = Trainer(model=aspect_model,
                      args=training_args,
                      train_dataset=tokenized_dataset_q_s["s"],
                      eval_dataset=tokenized_dataset_q_s["q"],
                      data_collator=data_collator,
                      tokenizer=tokenizer,
                      compute_metrics=compute_metrics)
    trainer.train()

silownie --------------------


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

loading weights file /content/drive/My Drive/clarin-biz/few-shot/checkpoint-444/pytorch_model.bin
All model checkpoint weights were used when initializing BertForTokenClassification.

All the weights of BertForTokenClassification were initialized from the model checkpoint at /content/drive/My Drive/clarin-biz/few-shot/checkpoint-444/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForTokenClassification for predictions without further training.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Overall Precision,Overall Recall,Overall F1,Overall Accuracy,Amb F1,Minus M F1,Minus S F1,Plus M F1,Plus S F1,Zero F1
1,No log,6.869126,0.032895,0.090909,0.048309,0.032432,0.000000,0.000000,0.000000,0.000000,0.000000,0.357143
2,No log,5.262764,0.038961,0.109091,0.057416,0.061261,0.000000,0.000000,0.000000,0.080000,0.000000,0.344828
3,No log,4.461326,0.037037,0.109091,0.055300,0.077477,0.000000,0.000000,0.000000,0.193548,0.000000,0.214286


...

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

loading weights file /content/drive/My Drive/clarin-biz/few-shot/checkpoint-444/pytorch_model.bin
All model checkpoint weights were used when initializing BertForTokenClassification.

All the weights of BertForTokenClassification were initialized from the model checkpoint at /content/drive/My Drive/clarin-biz/few-shot/checkpoint-444/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForTokenClassification for predictions without further training.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Overall Precision,Overall Recall,Overall F1,Overall Accuracy,Amb F1,Minus M F1,Minus S F1,Plus M F1,Plus S F1,Zero F1
1,No log,6.868132,0.045714,0.135593,0.068376,0.035385,0.039216,0.000000,0.166667,0.090909,0.000000,0.416667
2,No log,5.140156,0.040000,0.118644,0.059829,0.052308,0.000000,0.000000,0.166667,0.068966,0.000000,0.400000
3,No log,4.288873,0.044944,0.135593,0.067511,0.073846,0.000000,0.000000,0.100000,0.181818,0.000000,0.400000


The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10
  Batch size = 16


Training completed. Do 


# True few-shot learning

For this part, we need to change `tokenize_adjust_labels`. We need to add   `return_tensors=pt` in tokenizer's call:

In [ ]:
#Get the values for input_ids, token_type_ids, attention_mask
def tokenize_adjust_labels(all_samples_per_split):
  tokenized_samples = tokenizer(all_samples_per_split["tokens"], is_split_into_words=True, truncation=True, padding='max_length', return_tensors="pt")
  #tokenized_samples is not a datasets object so this alone won't work with Trainer API, hence map is used
  #so the new keys [input_ids, labels (after adjustment)]
  #can be added to the datasets dict for each train test validation split
  total_adjusted_labels = []
  for k in range(0, len(tokenized_samples["input_ids"])):
    prev_wid = -1
    word_ids_list = tokenized_samples.word_ids(batch_index=k)
    existing_label_ids = all_samples_per_split["labels"][k]
    i = -1
    adjusted_label_ids = []

    for wid in word_ids_list:
      if(wid is None):
        adjusted_label_ids.append(-100)
      elif(wid!=prev_wid):
        i = i + 1
        adjusted_label_ids.append(existing_label_ids[i])
        prev_wid = wid
      else:
        adjusted_label_ids.append(existing_label_ids[i])

    total_adjusted_labels.append(adjusted_label_ids)
  tokenized_samples["labels"] = total_adjusted_labels
  return tokenized_samples


Prepare samples for few show experiments, sample_size sets the size of Q and S sets, nr_samples is the number of samples in each set.

In [ ]:
import numpy as np

def sample_few_shot_data(df, sample_size, nr_samples):

  data_tuples = []
  cnt = 0
  toks = []; labels = []
  for index, row in df.iterrows():
    if pd.isna(row['token']):
      cnt += 1
      if len(toks)>0:
        assert len(labels) == len(toks)
        data_tuples.append( {'tokens':toks, 'labels':labels} )
      toks = []; labels = []
    else:
      toks.append( str(row['token']).strip() )
      labels.append( tag2id['O'] if pd.isna(row['label']) else tag2id[ str(row['label']).strip() ] )


  samples = []
  for _ in range(nr_samples):
    r = np.random.choice(data_tuples, sample_size*2, replace=False)
    s, q = np.split(r, 2)
    all_q = {}; all_s = {}
    all_q['tokens'] = [ x['tokens'] for x in q ]
    all_q['labels'] = [ x['labels'] for x in q ]
    all_s['tokens'] = [ x['tokens'] for x in s ]
    all_s['labels'] = [ x['labels'] for x in s ]

    samples.append( {'q':tokenize_adjust_labels(all_q), 's':tokenize_adjust_labels(all_s) } )
  return samples


Adaptation of prototypical networks (https://arxiv.org/pdf/1703.05175.pdf) for NER according to https://github.com/thunlp/Few-NERD/blob/main/model/proto.py

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np
import os
from torch import optim

class ProtoNetAspects(nn.Module):

    def __init__(self):

        nn.Module.__init__(self)
        model_directory = TODO_INSERT_YOUR_OWN
        BERT_MODEL="allegro/herbert-base-cased"
        MODEL_CLASSES = {"bert": (BertConfig, BertForTokenClassification, AutoTokenizer)}
        config_class, model_class, tokenizer_class = MODEL_CLASSES['bert']
        config = config_class.from_pretrained(BERT_MODEL, num_labels=len(id2tag), id2label=id2tag, label2id=tag2id )
        self.aspects_model = model_class.from_pretrained(model_directory,config=config)
        self.tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL, do_lower_case=False,  padding='max_length')
        self.dot = True


    def __dist__(self, x, y, dim):
        if self.dot:
            return (x * y).sum(dim)
        else:
            return -(torch.pow(x - y, 2)).sum(dim)

    def __batch_dist__(self, S, Q, q_mask):
        # S [class, embed_dim], Q [num_of_sent, num_of_tokens, embed_dim]
        assert Q.size()[:2] == q_mask.size()
        Q = Q[q_mask==1].view(-1, Q.size(-1)) # [num_of_all_text_tokens, embed_dim]
        return self.__dist__(S.unsqueeze(0), Q.unsqueeze(1), 2)

    def __get_proto__(self, embedding, tag, mask):
        proto = []
        embedding = embedding[mask>0,:]
        tag = tag[mask>0]

        assert tag.size(0) == embedding.size(0)
        for label in range(int(torch.max(tag))+1):
            proto.append(torch.mean(embedding[tag==label], 0))
        proto = torch.stack(proto)
        return proto


    def forward(self, support, query):

        support['input_ids']= support['input_ids'].squeeze()
        support['attention_mask']=support['attention_mask'].squeeze()
        support['token_type_ids']=support['token_type_ids'].squeeze()
        support['labels'] = torch.Tensor(support['labels'])

        query['input_ids']= query['input_ids'].squeeze()
        query['attention_mask']=query['attention_mask'].squeeze()
        query['token_type_ids']=query['token_type_ids'].squeeze()
        query['labels'] = torch.Tensor(query['labels'])

        support_emb = self.aspects_model(support['input_ids'],
                                         attention_mask=support['attention_mask'],
                                         token_type_ids=support['token_type_ids'],
                                         output_hidden_states=True)
        query_emb = self.aspects_model(query['input_ids'],
                                         attention_mask=query['attention_mask'],
                                         token_type_ids=query['token_type_ids'],
                                         output_hidden_states=True)

        # last hidden state of the transformer
        support_emb = support_emb.hidden_states[-1]
        query_emb = query_emb.hidden_states[-1]

        # Prototypical Networks
        logits = []

        assert support_emb.size()[:2] == support['attention_mask'].size()
        assert query_emb.size()[:2] == query['attention_mask'].size()

        # Calculate prototype for each class
        support_proto = self.__get_proto__( support_emb, support['labels'], support['attention_mask'])

        # calculate distance to each prototype
        logits.append(self.__batch_dist__(
                support_proto, query_emb,
                query['attention_mask'])) # [num_of_query_tokens, class_num]
        logits = torch.cat(logits, 0)
        _, pred = torch.max(logits, 1)
        return logits, pred


In [ ]:
from datasets import load_metric

class FewShotDataset(torch.utils.data.Dataset):
    def __init__(self, df, sample_size, nr_samples):
        # sample_few_shot_data(df, sample_size (of q / s), nr_samples)
        self.samples = sample_few_shot_data(df, sample_size, nr_samples)

    def __getitem__(self, index):
        return self.samples[index]

    def __len__(self):
        return len(self.samples)

all_results = {}
for domena, df in {'silownie':silownie_df, 'filmweb':filmweb_df, 'restauracje':restauracje_df}.items():

  print(domena)
  all_results[domena] = []
  few_shot_dataset = FewShotDataset(df, 10, 10) # nr_sentences (ineach of Q/S), nr_episodes
  few_shot_dataloader = torch.utils.data.DataLoader(few_shot_dataset, batch_size=1, shuffle=True, num_workers=0)

  model = ProtoNetAspects()

  for episode in few_shot_dataloader:

    logits, pred = model(episode['s'], episode['q'] )

    pred = pred.tolist()
    labels = episode['q']['labels'][ episode['q']['attention_mask']==1 ].tolist()

    # Remove ignored index (special tokens)
    true_predictions = [
          [label_names[int(p)] for (p, l) in zip([prediction], [label]) if l != -100]
          for prediction, label in zip(pred, labels)
    ]
    true_labels = [
          [label_names[int(l)] for (p, l) in zip([prediction], [label]) if l != -100]
          for prediction, label in zip(pred, labels)
    ]

    metric = load_metric("seqeval")
    results = metric.compute(predictions=true_predictions, references=true_labels)
    all_results[domena].append( results )


silownie


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: a_plus_m seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: a_plus_s seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: a_zero seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: a_minus_s seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: a_minus_m seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_l

filmweb
restauracje


In [ ]:
import numpy as np

def display_metrics(all_results):
  for domena, results in all_results.items():
    aggregated_res = {}
    for episode_res in results:
      for measure_name, values in episode_res.items():
        if measure_name not in aggregated_res: aggregated_res[measure_name] = []
        if type(values) is not dict:
          aggregated_res[measure_name].append( values )
        else:
          for m, v in values.items():
            if measure_name+'_'+m not in aggregated_res: aggregated_res[measure_name+'_'+m] = []
            aggregated_res[measure_name+'_'+m].append(v)

    print("-"*20)
    print(domena)
    for m, values in aggregated_res.items():
      print(f"{m} mean: {np.mean(values)}  std: {np.std(values)} ")

display_metrics(all_results)

--------------------
silownie
_amb mean: nan  std: nan 
_amb_precision mean: 0.16333333333333333  std: 0.18284784202536636 
_amb_recall mean: 0.4  std: 0.4358898943540674 
_amb_f1 mean: 0.23095238095238096  std: 0.2551176865950408 
_amb_number mean: 1.8  std: 1.077032961426901 
_minus_m mean: nan  std: nan 
_minus_m_precision mean: 0.293356845396319  std: 0.1628535751097343 
_minus_m_recall mean: 0.8261545643898586  std: 0.21601664217987082 
_minus_m_f1 mean: 0.40451457608498276  std: 0.16693127135690888 
_minus_m_number mean: 7.5  std: 4.201190307520001 
_minus_s mean: nan  std: nan 
_minus_s_precision mean: 0.5773267854150207  std: 0.1142492762667656 
_minus_s_recall mean: 0.556785665315077  std: 0.1202902752489195 
_minus_s_f1 mean: 0.5452571648624279  std: 0.07473028495336545 
_minus_s_number mean: 11.1  std: 3.6728735344413916 
_plus_m mean: nan  std: nan 
_plus_m_precision mean: 0.40878487395303864  std: 0.09434473335825301 
_plus_m_recall mean: 0.644806550762508  std: 0.10982843

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:263: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in true_divide
  subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



Implementation and tests of NNShot:

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np
import os
from torch import optim

class NNShotAspects(nn.Module):

    def __init__(self):

        nn.Module.__init__(self)
        model_directory = TODO_INSERT_YOUR_OWN
        BERT_MODEL="allegro/herbert-base-cased"
        MODEL_CLASSES = {"bert": (BertConfig, BertForTokenClassification, AutoTokenizer)}
        config_class, model_class, tokenizer_class = MODEL_CLASSES['bert']
        config = config_class.from_pretrained(BERT_MODEL, num_labels=len(id2tag), id2label=id2tag, label2id=tag2id )
        self.aspects_model = model_class.from_pretrained(model_directory,config=config)
        self.tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL, do_lower_case=False,  padding='max_length')
        self.dot = True


    def __dist__(self, x, y, dim):
        if self.dot:
            return (x * y).sum(dim)
        else:
            return -(torch.pow(x - y, 2)).sum(dim)

    def __batch_dist__(self, S, Q, q_mask):
        # S [class, embed_dim], Q [num_of_sent, num_of_tokens, embed_dim]
        assert Q.size()[:2] == q_mask.size()
        Q = Q[q_mask==1].view(-1, Q.size(-1)) # [num_of_all_text_tokens, embed_dim]
        return self.__dist__(S.unsqueeze(0), Q.unsqueeze(1), 2)

    def __get_proto__(self, embedding, tag, mask):
        proto = []
        embedding = embedding[mask>0,:]
        tag = tag[mask>0]

        assert tag.size(0) == embedding.size(0)
        for label in range(int(torch.max(tag))+1):
            proto.append(torch.mean(embedding[tag==label], 0))
        proto = torch.stack(proto)
        return proto

    def __get_nearest_dist__(self, embedding, tag, mask, query, q_mask):
        nearest_dist = []
        #S = embedding[mask==1].view(-1, embedding.size(-1))
        S = embedding[mask>0,:]
        #tag = torch.cat(tag, 0)
        tag = tag[mask>0]
        assert tag.size(0) == S.size(0)
        dist = self.__batch_dist__(S, query, q_mask) # [num_of_query_tokens, num_of_support_tokens]
        for label in range(int(torch.max(tag))+1):
            nearest_dist.append(torch.max(dist[:,tag==label], 1)[0])
        nearest_dist = torch.stack(nearest_dist, dim=1) # [num_of_query_tokens, class_num]
        return nearest_dist


    def forward(self, support, query):

        support['input_ids']= support['input_ids'].squeeze()
        support['attention_mask']=support['attention_mask'].squeeze()
        support['token_type_ids']=support['token_type_ids'].squeeze()
        support['labels'] = torch.Tensor(support['labels'])

        query['input_ids']= query['input_ids'].squeeze()
        query['attention_mask']=query['attention_mask'].squeeze()
        query['token_type_ids']=query['token_type_ids'].squeeze()
        query['labels'] = torch.Tensor(query['labels'])

        support_emb = self.aspects_model(support['input_ids'],
                                         attention_mask=support['attention_mask'],
                                         token_type_ids=support['token_type_ids'],
                                         output_hidden_states=True)
        query_emb = self.aspects_model(query['input_ids'],
                                         attention_mask=query['attention_mask'],
                                         token_type_ids=query['token_type_ids'],
                                         output_hidden_states=True)

        # last hidden state of the transformer
        support_emb = support_emb.hidden_states[-1]
        query_emb = query_emb.hidden_states[-1]

        # Prototypical Networks
        logits = []

        assert support_emb.size()[:2] == support['attention_mask'].size()
        assert query_emb.size()[:2] == query['attention_mask'].size()

        logits.append(self.__get_nearest_dist__(support_emb, support['labels'], support['attention_mask'], query_emb, query['attention_mask']))

        logits = torch.cat(logits, 0)
        _, pred = torch.max(logits, 1)
        return logits, pred


In [ ]:
from datasets import load_metric

class FewShotDataset(torch.utils.data.Dataset):
    def __init__(self, df, sample_size, nr_samples):
        # sample_few_shot_data(df, sample_size (of q / s), nr_samples)
        self.samples = sample_few_shot_data(df, sample_size, nr_samples)

    def __getitem__(self, index):
        return self.samples[index]

    def __len__(self):
        return len(self.samples)

all_results = {}
for domena, df in {'silownie':silownie_df, 'filmweb':filmweb_df, 'restauracje':restauracje_df}.items():

  print(domena)
  all_results[domena] = []
  few_shot_dataset = FewShotDataset(df, 10, 10) # nr_sentences (ineach of Q/S), nr_episodes
  few_shot_dataloader = torch.utils.data.DataLoader(few_shot_dataset, batch_size=1, shuffle=True, num_workers=0)

  model = NNShotAspects()

  for episode in few_shot_dataloader:

    logits, pred = model(episode['s'], episode['q'] )

    pred = pred.tolist()
    labels = episode['q']['labels'][ episode['q']['attention_mask']==1 ].tolist()

    # Remove ignored index (special tokens)
    true_predictions = [
          [label_names[int(p)] for (p, l) in zip([prediction], [label]) if l != -100]
          for prediction, label in zip(pred, labels)
    ]
    true_labels = [
          [label_names[int(l)] for (p, l) in zip([prediction], [label]) if l != -100]
          for prediction, label in zip(pred, labels)
    ]

    metric = load_metric("seqeval")
    results = metric.compute(predictions=true_predictions, references=true_labels)
    all_results[domena].append( results )


silownie


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: a_zero seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: a_minus_s seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: a_minus_m seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: a_plus_m seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: a_plus_s seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_l

filmweb


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


restauracje


In [ ]:
display_metrics(all_results)

--------------------
silownie
_amb mean: nan  std: nan 
_amb_precision mean: 0.24999999999999994  std: 0.292854723180093 
_amb_recall mean: 0.29500000000000004  std: 0.3044165640112976 
_amb_f1 mean: 0.1966017316017316  std: 0.15186820835826167 
_amb_number mean: 3.3  std: 1.4177446878757824 
_minus_m mean: nan  std: nan 
_minus_m_precision mean: 0.36077985207460916  std: 0.20350264781650124 
_minus_m_recall mean: 0.5419047619047619  std: 0.227941095147722 
_minus_m_f1 mean: 0.39027970452395006  std: 0.1781471957750038 
_minus_m_number mean: 6.9  std: 3.9862262855989496 
_minus_s mean: nan  std: nan 
_minus_s_precision mean: 0.5895959595959596  std: 0.1611162090324194 
_minus_s_recall mean: 0.4576334776334776  std: 0.2003323450696378 
_minus_s_f1 mean: 0.48540775401069514  std: 0.1460064409625824 
_minus_s_number mean: 10.5  std: 2.4186773244895647 
_plus_m mean: nan  std: nan 
_plus_m_precision mean: 0.3914181896811802  std: 0.09631792070331588 
_plus_m_recall mean: 0.4534888999888999

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:263: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in true_divide
  subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
